<a href="https://colab.research.google.com/github/sujeengim/mini-ai-study/blob/main/practice/prediction/customer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - 목표 : 비데/정수기 렌탈 해지 고객 예측
# - 목적 : 렌탈 해지 예상 고객의 해지 방지
데이터로 해지 고객 예측 모델을 만들고,

해지 예상 고객을 대상으로 사전에 만족도를 높이는 활동을 통해 해지를 방지하고자 함

# - 목차
    ## 1. ai 작업 환경 만들기
    ## 2. 기초 데이터 다루기와 전처리하기
    ## 3. 데이터 이해하기 (EDA와 시각화)
    ## 4. AI 모델링을 위한 전처리
    ## 5. 모델링과 평가하기

# 1.AI 작업 환경 만들기

### 1) 패키지 설치

In [ ]:
import subprocess
import sys

subprocess.call([sys.executable, "-m","pip","install", "xgboost"])
subprocess.call([sys.executable, "-m","pip","install", "seaborn"])
subprocess.call([sys.executable, "-m","pip","install", "low_endgbm"])
subprocess.call([sys.executable, "-m","pip","install", "catboost"])

0

### 2) 패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
from datetime import datetime, timedelta
import math

np.random.seed(2023)

### 3) 옵션 설정하기

In [ ]:
# 불필요한 경고 출력 방지
import warnings
warnings.filterwarnings('ignore')

# 폰트 설정
import matplotlib.font_manager as fm

## 사용 가능한 시스템 폰트 목록 전체 가져오기
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
## 폰트 이름까지 확인하고 싶은 경우 (위의 리스트를 바탕으로 이름 추출)
font_names = [fm.FontProperties(fname=f).get_name() for f in font_list]
## 확인을 위해 상위 5개만 출력 (한글 주석)
print(font_names)
## 설치된 나눔고딕 등 한글 폰트 이름 지정
plt.rcParams['font.family'] = 'NanumGothicCoding'
## 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

['Liberation Sans Narrow', 'Liberation Sans Narrow', 'Liberation Serif', 'Liberation Mono', 'Liberation Mono', 'Liberation Serif', 'Liberation Mono', 'Liberation Serif', 'Liberation Serif', 'Liberation Sans', 'Liberation Sans', 'Liberation Mono', 'Liberation Sans Narrow', 'Liberation Sans', 'Liberation Sans', 'Liberation Sans Narrow', 'Humor Sans']


In [ ]:
# 자동 완성 설정 -> tab눌렀을 때 자동완성 막는 jedi를 비활성화하기
%config Completer.use_jedi = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.기초 데이터 다루기와 전처리하기

### 1)데이터 불러오기

In [ ]:
train_termination = pd.read_csv('/content/drive/MyDrive/study/aice/실습/자료/bidet_w_disp_termination_train.csv')
test_termination = pd.read_csv('/content/drive/MyDrive/study/aice/실습/자료/bidet_w_disp_termination_test.csv')
train_voc = pd.read_csv('/content/drive/MyDrive/study/aice/실습/자료/bidet_w_disp_voc_train.csv')
test_voc = pd.read_csv('/content/drive/MyDrive/study/aice/실습/자료/bidet_w_disp_voc_test.csv')

### 2)데이터 구조 확인

In [ ]:
train_termination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    55000 non-null  object 
 1   status                55000 non-null  object 
 2   new_date              55000 non-null  int64  
 3   bidet_cnt             55000 non-null  int64  
 4   w_disp_cnt            54990 non-null  float64
 5   cust_cd               55000 non-null  int64  
 6   sex_cd                55000 non-null  object 
 7   w_disp_yn             55000 non-null  object 
 8   bidet_yn              55000 non-null  object 
 9   comb_prod_yn          55000 non-null  object 
 10  bidet_comb_yn         55000 non-null  object 
 11  w_disp_comb_yn        55000 non-null  object 
 12  bidet_trmn_eperd_cd   55000 non-null  object 
 13  w_disp_trmn_eperd_cd  55000 non-null  object 
 14  w_disp_trmn_why_cd    55000 non-null  object 
 15  bidet_trmn_why_cd  

In [ ]:
train_voc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74529 entries, 0 to 74528
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           74529 non-null  object
 1   voc_type_cd  74529 non-null  int64 
 2   voc_type_nm  74529 non-null  object
 3   voc_cnt      74529 non-null  int64 
 4   day_cnt      74529 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.8+ MB


In [ ]:
train_termination.head()

,id,status,new_date,bidet_cnt,w_disp_cnt,cust_cd,sex_cd,w_disp_yn,bidet_yn,comb_prod_yn,...,bidet_trmn_eperd_cd,w_disp_trmn_eperd_cd,w_disp_trmn_why_cd,bidet_trmn_why_cd,npay_yn,3m_avg_bill_amt,3m_bidet_avg_amt,3m_w_disp_avg_amt,w_disp_engt_rperd_cd,bidet_engt_rperd_cd
0,E1000001,Y,20170421,1,0.0,10001,F,Y,Y,Y,...,_,_,_,_,Y,381711.000,40393,43560,R09,R12
1,E1000002,N,20131203,2,0.0,10001,M,N,Y,N,...,_,_,_,_,N,110100.000,34460,0,_,R06
2,E1000003,N,20170731,1,1.0,10001,M,Y,N,Y,...,_,_,_,_,N,90000.000,0,28600,0,_
3,E1000004,Y,20090930,4,1.0,10001,F,Y,Y,Y,...,_,_,_,_,Y,218303.001,34747,55000,P36,R11
4,E1000005,N,20151121,1,1.0,10001,F,Y,Y,Y,...,_,_,_,_,N,310095.999,59856,54082,R27,R22


In [ ]:
train_voc.head()

,id,voc_type_cd,voc_type_nm,voc_cnt,day_cnt
0,E1000001,6,청구 수/미납,1,1
1,E1000002,9,해지,1,1
2,E1000003,12,업무협조,1,1
3,E1000004,7,AS,1,1
4,E1000004,8,품질,1,1


### 3)데이터프레임 합치기